In [1]:
pip install jieba

Note: you may need to restart the kernel to use updated packages.


In [2]:
import nltk
from nltk.corpus import stopwords
import string
import pandas as pd
import re
import jieba
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

In [3]:
import chardet
with open('ICDM_REVIEWS_TO_RELEASE_encoding=utf-8.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}

In [4]:
def find_chinese(file):
    pattern = re.compile(r'[^\u4e00-\u9fa5]')
    chinese_txt = re.sub(pattern,'',file)
    return chinese_txt

In [5]:
rawData = open("ICDM_REVIEWS_TO_RELEASE_encoding=utf-8.csv",encoding='utf-8').read()

# Print the raw data
rawData[0:700]

'review_id,label,user,ip,star,text\nREVIEW0,+,USER1,IP1,5,他家的面很有嚼劲，牛肉汤很有味道，服务员的服务也特别的好，其中服务员顾存芳的服务特好！好喜欢她给我们服务！！！！\nREVIEW1,-,USER2,IP2,4,鲜榨果汁很不错，水果都是很新鲜的，口感也很好，是个小妹妹态度非常好，环境也很好，但是专家告诉我们还是吃水果比较好  嘿嘿\nREVIEW2,+,USER3,IP3,5,跟老婆过二人世界，就定了他家的一间包房，他家挺好的，跟老婆点了几个菜，边吃边聊，包房的话，那个小姑娘看我们两人也没怎么打扰，着让我感到挺舒服的\nREVIEW3,-,USER4,IP4,5,我是在开业那天去得，他们的环境很好，进去给人一种很温软的感觉  他们的服务很好，销售很耐心的跟我和我老公介绍了各种口味  爆米花价格性价比很高，口味很多。  总得来说他们的服务态度非常的好，价格又很实惠，建议喜欢的朋友可以沃尔玛买~\nREVIEW4,-,USER5,IP5,4,好像说属于宁波菜系，东西还可以，只是上菜速度比较慢，好在服务还不错。还有赠送的餐前小吃，水果是餐前送的，这一点很科学，赞！\nREVIEW5,+,USER6,IP6,3,今天过来看一下，顺便买了一些零食，好实惠，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，服务员很热情。\nREVIEW6,-,USER7,IP7,5,2号线科技馆下来找了好久，离地铁还是蛮远的，不过面包真心赞啊！有机会真想把这儿的面包全买了。环境也不错，价格也没有很离谱啦！ 红酒芝士的芝士真是超级多，满满的+_+\nR'

In [6]:
data = pd.read_csv('ICDM_REVIEWS_TO_RELEASE_encoding=utf-8.csv',encoding='utf-8', 
                   usecols=[1,5], names=['tag','text'], header=0)
print(data['text'])
print('-----------------------------------------------------------')
print(data.head())

0       他家的面很有嚼劲，牛肉汤很有味道，服务员的服务也特别的好，其中服务员顾存芳的服务特好！好喜欢...
1       鲜榨果汁很不错，水果都是很新鲜的，口感也很好，是个小妹妹态度非常好，环境也很好，但是专家告诉...
2       跟老婆过二人世界，就定了他家的一间包房，他家挺好的，跟老婆点了几个菜，边吃边聊，包房的话，那...
3       我是在开业那天去得，他们的环境很好，进去给人一种很温软的感觉  他们的服务很好，销售很耐心的...
4       好像说属于宁波菜系，东西还可以，只是上菜速度比较慢，好在服务还不错。还有赠送的餐前小吃，水果...
                              ...                        
9760    之前一直在苏浙汇吃，看到这边新开了家丰收日就去尝了下，味道不错，价格也比较实惠，不是很贵，包...
9761    我一直等着世界末日的到来，等到21号，钱用光了，什么都没有了，结果它没有来，TMD谁造的谣啊...
9762         菜很好吃，尤其是剁椒鱼头，强烈推荐！！建议老板稍微扩大一下场地，位置太少！每次去都要等！
9763                     公司楼下 最好吃的麻辣烫 老板是河南人 煮麻辣烫的小姑娘长得不错
9764    经常去的鸭王，每天每桌还可以点两道特价菜。很实惠，菜的味道又好。我还经常给我周边的朋友介绍他...
Name: text, Length: 9765, dtype: object
-----------------------------------------------------------
  tag                                               text
0   +  他家的面很有嚼劲，牛肉汤很有味道，服务员的服务也特别的好，其中服务员顾存芳的服务特好！好喜欢...
1   -  鲜榨果汁很不错，水果都是很新鲜的，口感也很好，是个小妹妹态度非常好，环境也很好，但是专家告诉...
2   +  跟老婆过二人世界，就定了他家的一间包房，他家挺好的，跟老婆点了几个菜，边吃边聊，包房的话，那...
3   -  我是在开业那天去得，他们的环境很好，进去给人一种很温软

In [7]:
print('data.shape:',data.shape)
print('type(data)', type(data))

data.shape: (9765, 2)
type(data) <class 'pandas.core.frame.DataFrame'>


In [8]:
labelList = []
textList = []
print(data['tag'])
for i in data['tag']:
    labelList.append(i)
for j in data['text']:
    textList.append(j)

print(labelList[0:5])
print(textList[0:5])

print(len(labelList))
print(len(textList))
print("Input data has {} rows and {} columns".format(len(data), len(data.columns)))

0       +
1       -
2       +
3       -
4       -
       ..
9760    +
9761    -
9762    +
9763    -
9764    +
Name: tag, Length: 9765, dtype: object
['+', '-', '+', '-', '-']
['他家的面很有嚼劲，牛肉汤很有味道，服务员的服务也特别的好，其中服务员顾存芳的服务特好！好喜欢她给我们服务！！！！', '鲜榨果汁很不错，水果都是很新鲜的，口感也很好，是个小妹妹态度非常好，环境也很好，但是专家告诉我们还是吃水果比较好  嘿嘿', '跟老婆过二人世界，就定了他家的一间包房，他家挺好的，跟老婆点了几个菜，边吃边聊，包房的话，那个小姑娘看我们两人也没怎么打扰，着让我感到挺舒服的', '我是在开业那天去得，他们的环境很好，进去给人一种很温软的感觉  他们的服务很好，销售很耐心的跟我和我老公介绍了各种口味  爆米花价格性价比很高，口味很多。  总得来说他们的服务态度非常的好，价格又很实惠，建议喜欢的朋友可以沃尔玛买~', '好像说属于宁波菜系，东西还可以，只是上菜速度比较慢，好在服务还不错。还有赠送的餐前小吃，水果是餐前送的，这一点很科学，赞！']
9765
9765
Input data has 9765 rows and 2 columns


In [9]:
pip install zhon

Note: you may need to restart the kernel to use updated packages.


In [10]:
from zhon.hanzi import punctuation
print(punctuation)

＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､　、〃〈〉《》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏﹑﹔·！？｡。


In [11]:
def remove_punct(text):
    text_only = re.sub('[{}]+'.format(punctuation), "", text)
    return text_only
data['text_nopunct'] = data['text'].apply(lambda x: remove_punct(x))
print(data['text_nopunct'])

0        他家的面很有嚼劲牛肉汤很有味道服务员的服务也特别的好其中服务员顾存芳的服务特好好喜欢她给我们服务
1       鲜榨果汁很不错水果都是很新鲜的口感也很好是个小妹妹态度非常好环境也很好但是专家告诉我们还是吃...
2       跟老婆过二人世界就定了他家的一间包房他家挺好的跟老婆点了几个菜边吃边聊包房的话那个小姑娘看我...
3       我是在开业那天去得他们的环境很好进去给人一种很温软的感觉  他们的服务很好销售很耐心的跟我和...
4       好像说属于宁波菜系东西还可以只是上菜速度比较慢好在服务还不错还有赠送的餐前小吃水果是餐前送的...
                              ...                        
9760    之前一直在苏浙汇吃看到这边新开了家丰收日就去尝了下味道不错价格也比较实惠不是很贵包房相对比较...
9761    我一直等着世界末日的到来等到21号钱用光了什么都没有了结果它没有来TMD谁造的谣啊我就剩10...
9762                菜很好吃尤其是剁椒鱼头强烈推荐建议老板稍微扩大一下场地位置太少每次去都要等
9763                     公司楼下 最好吃的麻辣烫 老板是河南人 煮麻辣烫的小姑娘长得不错
9764    经常去的鸭王每天每桌还可以点两道特价菜很实惠菜的味道又好我还经常给我周边的朋友介绍他们家呢这...
Name: text_nopunct, Length: 9765, dtype: object


In [12]:
def tokenize(text):
    jieba_list = jieba.cut(text, cut_all=False)
    tokens = ' '.join(jieba_list).split(' ')
    return tokens
data['text_tokenized'] = data['text_nopunct'].apply(lambda x:tokenize(x.lower()))
data.head()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\liqi9\AppData\Local\Temp\jieba.cache
Loading model cost 0.596 seconds.
Prefix dict has been built successfully.


,tag,text,text_nopunct,text_tokenized
0,+,他家的面很有嚼劲，牛肉汤很有味道，服务员的服务也特别的好，其中服务员顾存芳的服务特好！好喜欢...,他家的面很有嚼劲牛肉汤很有味道服务员的服务也特别的好其中服务员顾存芳的服务特好好喜欢她给我们服务,"[他家, 的, 面, 很, 有, 嚼, 劲, 牛肉汤, 很, 有, 味道, 服务员, 的, ..."
1,-,鲜榨果汁很不错，水果都是很新鲜的，口感也很好，是个小妹妹态度非常好，环境也很好，但是专家告诉...,鲜榨果汁很不错水果都是很新鲜的口感也很好是个小妹妹态度非常好环境也很好但是专家告诉我们还是吃...,"[鲜榨, 果汁, 很, 不错, 水果, 都, 是, 很, 新鲜, 的, 口感, 也, 很, ..."
2,+,跟老婆过二人世界，就定了他家的一间包房，他家挺好的，跟老婆点了几个菜，边吃边聊，包房的话，那...,跟老婆过二人世界就定了他家的一间包房他家挺好的跟老婆点了几个菜边吃边聊包房的话那个小姑娘看我...,"[跟, 老婆, 过, 二人, 世界, 就定, 了, 他家, 的, 一间, 包房, 他家, 挺..."
3,-,我是在开业那天去得，他们的环境很好，进去给人一种很温软的感觉 他们的服务很好，销售很耐心的...,我是在开业那天去得他们的环境很好进去给人一种很温软的感觉 他们的服务很好销售很耐心的跟我和...,"[我, 是, 在, 开业, 那天, 去, 得, 他们, 的, 环境, 很, 好, 进去, 给..."
4,-,好像说属于宁波菜系，东西还可以，只是上菜速度比较慢，好在服务还不错。还有赠送的餐前小吃，水果...,好像说属于宁波菜系东西还可以只是上菜速度比较慢好在服务还不错还有赠送的餐前小吃水果是餐前送的...,"[好像, 说, 属于, 宁波, 菜系, 东西, 还, 可以, 只是, 上菜, 速度, 比较慢..."


In [13]:
with open('stopwords.txt','r', encoding='GB18030')as s:
    stopwords = s.read().splitlines()
print(stopwords)

['一', '一.', '一一', '一下', '一个', '一些', '一何', '一切', '一则', '一则通过', '一天', '一定', '一方面', '一旦', '一时', '一来', '一样', '一次', '一片', '一番', '一直', '一致', '一般', '一起', '一转眼', '一边', '一面', '七', '万一', '三', '三天两头', '三番两次', '三番五次', '上', '上下', '上升', '上去', '上来', '上述', '上面', '下', '下列', '下去', '下来', '下面', '不', '不一', '不下', '不久', '不了', '不亦乐乎', '不仅', '不仅...而且', '不仅仅', '不仅仅是', '不会', '不但', '不但...而且', '不光', '不免', '不再', '不力', '不单', '不变', '不只', '不可', '不可开交', '不可抗拒', '不同', '不外', '不外乎', '不够', '不大', '不如', '不妨', '不定', '不对', '不少', '不尽', '不尽然', '不巧', '不已', '不常', '不得', '不得不', '不得了', '不得已', '不必', '不怎么', '不怕', '不惟', '不成', '不拘', '不择手段', '不敢', '不料', '不断', '不日', '不时', '不是', '不曾', '不止', '不止一次', '不比', '不消', '不满', '不然', '不然的话', '不特', '不独', '不由得', '不知不觉', '不管', '不管怎样', '不经意', '不胜', '不能', '不能不', '不至于', '不若', '不要', '不论', '不起', '不足', '不过', '不迭', '不问', '不限', '与', '与其', '与其说', '与否', '与此同时', '专门', '且', '且不说', '且说', '两者', '严格', '严重', '个', '个人', '个别', '中小', '中间', '丰富', '串行', '临', '临到', '为', '为主', '为了', '为什么', '为什麽', '为何', '为止', '为此', '为着', '主张', '

In [14]:
def remove_stopwords(tokenized_list):
    text = [word for word in tokenized_list if word not in stopwords]
    return text
data['text_nonstop'] = data['text_tokenized'].apply(lambda x: remove_stopwords(x))
data['text_nonstop']

0       [他家, 面, 嚼, 劲, 牛肉汤, 味道, 服务员, 服务, 特别, 服务员, 顾存芳, ...
1       [鲜榨, 果汁, 不错, 水果, 新鲜, 口感, 小妹妹, 态度, 环境, 专家, 告诉, ...
2       [老婆, 二人, 世界, 就定, 他家, 一间, 包房, 他家, 挺, 老婆, 点, 几个,...
3       [开业, 那天, 环境, 一种, 温软, 感觉, 服务, 销售, 耐心, 老公, 介绍, 口...
4       [好像, 说, 宁波, 菜系, 东西, 上菜, 速度, 比较慢, 服务, 不错, 赠送, 餐...
                              ...                        
9760    [苏浙, 汇吃, 新开, 家, 丰收, 日, 尝, 味道, 不错, 价格, 实惠, 贵, 包...
9761    [世界末日, 到来, 21, 号, 钱, 用光, tmd, 谁造, 谣, 剩, 100, 钱...
9762    [菜, 好吃, 剁, 椒, 鱼头, 强烈推荐, 建议, 老板, 稍微, 场地, 位置, 太,...
9763      [公司, 楼下, 好吃, 麻辣烫, 老板, 河南人, 煮, 麻辣烫, 小姑娘, 长得, 不错]
9764    [鸭, 王, 每桌, 点, 两道, 特价, 菜, 实惠, 菜, 味道, 周边, 朋友, 介绍...
Name: text_nonstop, Length: 9765, dtype: object

In [20]:
def clean_text(text):
    text_only = re.sub('[{}]+'.format(punctuation), "", text)
    jieba_list = jieba.cut(text_only, cut_all=False)
    tokens = ' '.join(jieba_list).split(' ')   
    text_nonstop = [word for word in tokens if word not in stopwords]
    return text_nonstop

data['clean'] = data['text'].apply(lambda x: clean_text(x))

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['text'])
print(X_tfidf.shape)
print(tfidf_vect.get_feature_names())

AttributeError: 'list' object has no attribute 'lower'